In [0]:
%run "/Workspace/Projects/formula1/Data Ingestion/Prod/01-setup"

In [0]:
%run "/Workspace/Projects/formula1/Data Ingestion/Prod/02-common_functions"

Read json file

In [0]:
from pyspark.sql.types import StructType, StructField, IntegerType, StringType, DateType


In [0]:
name_schema= StructType(fields=[StructField('forename', StringType(), True),
                                StructField('surname', StringType(), True)
                         ])


In [0]:
drivers_schema=StructType(fields=[StructField('driverId', IntegerType(), False),
                                StructField('driverRef', StringType(), True),
                                StructField('number', IntegerType(), True),
                                StructField('code', StringType(), True),
                                StructField('name', name_schema),
                                StructField('dob', DateType(), True),
                                StructField('nationality', StringType(), True),
                                StructField('url', StringType(), True)
                         ])

In [0]:
drivers_df=spark.read.json(f'{raw_folder_path}/drivers.json',schema=drivers_schema)


In [0]:
from pyspark.sql import functions
from pyspark.sql.functions import col, current_timestamp, lit, concat

In [0]:
divers_renamed_df=drivers_df.withColumnRenamed('driverId','driver_id')\
                            .withColumnRenamed('driverRef','driver_ref')\
                            .withColumn('Ingestion_date',current_timestamp())\
                            .withColumn('name',concat(col('name.forename'),lit(' '),col('name.surname')))\
                            .withColumn("data_source",lit("v_data_source"))
                            .drop('url')
                            

In [0]:
divers_renamed_df.write.mode('overwrite').parquet(f'{processed_folder_path}/drivers')

In [0]:
dbutils.notebook.exit("Succeed")